Import Libraries

In [1]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


In [4]:
trips.dtypes

trip_id                 int64
subscriber_type        object
bikeid                 object
start_time             object
start_station_id      float64
start_station_name     object
end_station_id        float64
end_station_name       object
duration_minutes        int64
dtype: object

**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [5]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [3]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

In [4]:
conn

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [8]:
# Get the data values
data = tuple(stations.iloc[19].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [9]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (1008, 'Nueces @ 3rd', 'closed', '311 Nueces', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [10]:
# Execute the query 
conn.execute(query)

In [11]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [12]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [13]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[14].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [14]:
result

'OK'

**TASK: Create a Function to insert data for `trips` table**

In [17]:
# Your code here
def insert_into_trips(data_trips, conn):
    query = f"""INSERT INTO trips values {data_trips}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

conn = make_connection()
data_trips = tuple(trips.iloc[9].fillna('').values) # Randomly select a data
result_trips = insert_into_trips(data_trips, conn)

In [18]:
result_trips

'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [53]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [54]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [55]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [56]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [78]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

In [21]:
get_station_id(2550, conn)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Create a Function to read data from `trips` table**

In [62]:
# create a function to know the longest (trip in minute)
def get_longest_duration_trip(conn):
    query = f"""SELECT duration_minutes FROM trips ORDER by duration_minutes DESC LIMIT 1"""
    result = pd.read_sql_query(query, conn)
    return result
longest_duration_trip

<function __main__.longest_duration_trip(conn)>

In [61]:
query = f"""SELECT duration_minutes FROM trips ORDER by duration_minutes DESC LIMIT 1 """
result = pd.read_sql_query(query, conn)
result

,duration_minutes
0,34238


In [24]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""SELECT * FROM trips WHERE id = {trip_id}"""
    result = pd.read_sql_query(query, conn)
    return result
    
def get_all_trips(conn):
    query = f""" SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result


In [25]:
get_trip_id(8269945, conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,8269945,Walk Up,311,2016-01-01 00:43:41 UTC,2707,Rainey St @ Cummings,2502,Barton Springs & Riverside,20


In [ ]:
get_all_trips(conn)

# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [ ]:
# your code

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [26]:
url = 'http://127.0.0.1:5000/trips/'
res = requests.get(url)

In [27]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [28]:
pd.DataFrame(res.json())#.head()

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
...,...,...,...,...,...,...,...,...,...
1324081,22837019,Local365,19660,2020-09-17 14:21:51 UTC,3838,26th/Nueces,4061,Lakeshore/Austin Hostel,68
1324082,23184844,Local365,19427,2020-11-13 17:02:07 UTC,3838,26th/Nueces,4061,Lakeshore/Austin Hostel,40
1324083,23184851,Local365,16456,2020-11-13 17:02:51 UTC,3838,26th/Nueces,4061,Lakeshore/Austin Hostel,40
1324084,23369810,Local365,451,2021-01-01 12:08:32 UTC,2823,East 5th/Broadway @ Capital Metro HQ,4062,Lakeshore/Pleasant Valley,13


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [73]:
station_id = 2500
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [74]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

In [31]:
trip_id = 8269930
url = f"http://127.0.0.1:5000/trips/{trip_id}"

In [32]:
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,8269930,Local365,460,2016-01-01 00:18:49 UTC,1007,Lavaca & 6th,2501,5th & Bowie,4


## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [33]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [34]:
res = requests.post(url, json=data)

In [35]:
res

<Response [200]>

In [36]:
res.text

'Hello Andi, your age is 24, and your address in my adress\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [38]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [45]:
data = trips.iloc[13].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [40]:
stations.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
1,2500,Republic Square,closed,425 W 4th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
2,2536,Waller & 6th St.,closed,602 Waller St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2021-01-04T12:00:00Z
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z


In [91]:
url = "http://127.0.0.1:5000/trips/average_duration"
res = requests.get(url)

In [92]:
res

<Response [200]>

In [93]:
res.text

'{"duration_minutes":30.5868362659}'

In [44]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:02, 46.42it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [49]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

17982it [07:42, 38.85it/s]


In [50]:
res

<Response [200]>

In [52]:
res.text

'OK'

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint

In [89]:
conn = make_connection()
query = """SELECT duration_minutes FROM trips"""
result = pd.read_sql_query(query, conn).mean()
result

duration_minutes    30.586836
dtype: float64

In [ ]:
# Your Code Here 
@app.route('/trips/average_duration') 

In [ ]:
# create a function to know the longest (trip in minute)
def get_longest_duration_trip(conn):
    query = f"""SELECT duration_minutes FROM trips ORDER by duration_minutes DESC LIMIT 1"""
    result = pd.read_sql_query(query, conn)
    return result
longest_duration_trip

In [115]:
# create a function to know subscriber type 
conn = make_connection()

def get_subscriber_type(conn):
    query = """SELECT subscriber_type, duration_minutes FROM trips"""
    selected_data = pd.read_sql_query(query, conn)
    result = selected_data.groupby('subscriber_type').agg(total_trip = ('subscriber_type', 'count'),
                                                     total_duration = ('duration_minutes','sum')).sort_values(by='total_trip', ascending = False)
    return result


,total_trip,total_duration
subscriber_type,,
Walk Up,374151,15928365
U.T. Student Membership,287267,5155436
Local365,236189,3801752
24-Hour Kiosk (Austin B-cycle),108672,4062151
Local30,69062,1234248
...,...,...
Local365 Youth (age 13-17 riders)- 1/2 off Special,3,21
Annual Pass (30 minute),2,17
Heartland Pass (Annual Pay),1,47


In [109]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342066 entries, 0 to 1342065
Data columns (total 2 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   subscriber_type   1342066 non-null  object
 1   duration_minutes  1342066 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 20.5+ MB


## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint

In [ ]:
# Your Code Here
@app.route('/trips/average_duration/<bike_id>')

In [63]:
# create a function to know the longest trip of a specific bikeid (in minute)
def get_longest_duration_trip_bikeid(bikeid, conn):
    query = f"""SELECT duration_minutes FROM trips WHERE bikeid = {bikeid} ORDER by duration_minutes DESC LIMIT 1"""
    result = pd.read_sql_query(query, conn)
    return result
get_longest_duration_trip_bikeid

<function __main__.get_longest_duration_trip_id(bikeid, conn)>

In [69]:
trips.head()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16
2,23436548,Local365,18331,2021-01-21 07:09:48 UTC,4055.0,11th/Salina,4055.0,11th/Salina,25
3,23382542,Local365,19682,2021-01-04 09:38:18 UTC,4062.0,Lakeshore/Pleasant Valley,3293.0,East 2nd/Pedernales,5
4,23374882,Local365,19437,2021-01-02 15:04:01 UTC,3293.0,East 2nd/Pedernales,4062.0,Lakeshore/Pleasant Valley,5


In [72]:
stations.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
1,2500,Republic Square,closed,425 W 4th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
2,2536,Waller & 6th St.,closed,602 Waller St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2021-01-04T12:00:00Z
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z


In [101]:
url = "http://127.0.0.1:5000/trips/average_trip_duration/174"
res = requests.get(url)

In [103]:
res

<Response [200]>

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

In [8]:
# coba
conn = make_connection()
query = f"""SELECT * FROM trips WHERE start_time LIKE '2018-08%'"""
selected_data = pd.read_sql_query(query, conn)

In [18]:
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

In [19]:
result.head(10)

,bikeid,duration_minutes
start_station_id,,
2494,263,34.448669
2495,185,38.745946
2496,112,23.196429
2497,237,44.818565
2498,186,31.994624
2499,255,27.400000
2501,286,33.290210
2502,161,32.372671
2503,162,42.401235


In [21]:
result2 = selected_data.groupby('start_station_id').agg(
    bike_count = ('bikeid' , 'count'), 
    average_duration = ('duration_minutes' , 'mean')
)

In [22]:
result2.head(10)

,bike_count,average_duration
start_station_id,,
2494,263,34.448669
2495,185,38.745946
2496,112,23.196429
2497,237,44.818565
2498,186,31.994624
2499,255,27.400000
2501,286,33.290210
2502,161,32.372671
2503,162,42.401235


In [116]:
data = { "period" : "2015-08" }

url = "http://127.0.0.1:5000/trips/summary_by_date"

In [117]:
res = requests.post(url, json=data)

In [118]:
res

<Response [200]>

In [119]:
res.text

'{"bike_count":{"1007":91,"1008":311,"2494":791,"2495":590,"2496":250,"2497":440,"2498":427,"2499":635,"2501":694,"2502":422,"2503":292,"2504":213,"2536":106,"2537":290,"2538":229,"2539":394,"2540":173,"2541":84,"2542":147,"2544":122,"2545":46,"2547":119,"2548":199,"2549":297,"2550":296,"2552":430,"2561":128,"2562":193,"2563":260,"2564":308,"2565":249,"2566":503,"2567":306,"2568":112,"2569":210,"2570":321,"2571":150,"2572":314,"2575":643,"2576":226,"2707":655,"2711":501,"2712":241,"2822":189,"2823":58,"3291":50,"3292":71,"3293":40,"":707},"average_duration":{"1007":18.2307692308,"1008":13.2379421222,"2494":26.0341340076,"2495":15.4593220339,"2496":27.624,"2497":27.4136363636,"2498":22.018735363,"2499":18.0960629921,"2501":15.476945245,"2502":31.2819905213,"2503":29.0719178082,"2504":21.9530516432,"2536":32.1509433962,"2537":12.1206896552,"2538":25.3886462882,"2539":22.9035532995,"2540":26.9190751445,"2541":22.75,"2542":14.5238095238,"2544":14.8442622951,"2545":11.2608695652,"2547":52.4

In [38]:
# Buat sendiri
data = {"period" : "2016-01",
        "start_station" : "Rosewood/Chicon"}

In [41]:
conn = make_connection()
query = f"""SELECT * FROM trips WHERE start_time LIKE '2016-01%' AND start_station_name LIKE 'Lavaca & 6th'"""
selected_data = pd.read_sql_query(query, conn)

In [42]:
result = selected_data.groupby('bikeid').agg(
    bike_count = ('bikeid' , 'count'), 
    average_duration = ('duration_minutes' , 'mean')
)
result.head()

,bike_count,average_duration
bikeid,,
101,6,6.333333
104,2,20.000000
122,2,20.000000
124,2,22.000000
132,2,2.000000


In [54]:
# coba stasion
conn = make_connection()
query = f"""SELECT * FROM trips WHERE start_station_name LIKE 'Lavaca & 6th'"""
selected_data = pd.read_sql_query(query, conn)

In [59]:
result = selected_data.groupby('bikeid').agg(
    bike_used_in_specific_station = ('bikeid' , 'count'), 
    average_duration = ('duration_minutes' , 'mean')
)
result

,bike_used_in_specific_station,average_duration
bikeid,,
009G,1,19.000000
034G,1,22.000000
035G,2,524.000000
040G,1,29.000000
041G,2,15.500000
...,...,...
993,14,21.000000
995,18,20.277778
996,27,18.074074


In [128]:
data = {"start_station" : "Lavaca & 6th"}

url = "http://127.0.0.1:5000/trips/summary_by_station"

In [129]:
res = requests.post(url, json=data)
res

<Response [200]>

In [130]:
res.text

'{"bike_count_start_from_the_station":{"009G":1,"034G":1,"035G":2,"040G":1,"041G":2,"046G":1,"050G":1,"062G":3,"066G":1,"072G":1,"080G":2,"086G":1,"095G":1,"100":10,"1000":15,"101":19,"102":14,"104":19,"105":13,"105G":1,"106":15,"106G":2,"107":12,"109":2,"110":7,"113":6,"114":10,"115":6,"116":16,"117":11,"122":21,"1224":4,"123":5,"124":24,"126G":1,"127":8,"12740":3,"1277":5,"12802":1,"12811":3,"132":19,"133":8,"133G":1,"134":20,"135":23,"136G":3,"1378":5,"1386":5,"138G":1,"14":7,"140":17,"141":11,"14143":1,"14151":1,"142":17,"1423":6,"1424":2,"14251":1,"14264":2,"1434":3,"1439":4,"1441":6,"1463":5,"1471":5,"1475":6,"1484":7,"1486":7,"1487":4,"148G":2,"1492":5,"15":8,"150":18,"15026E":1,"15027E":5,"15029E":1,"15030E":2,"15031E":2,"15040E":1,"15056E":2,"15059E":1,"15063E":1,"15074E":1,"1511":4,"1517":1,"152":15,"1527":2,"153":10,"1531":7,"1543":7,"1545":1,"1551":2,"1554":4,"1555":4,"1558":3,"156":6,"1565":2,"157":14,"1574":4,"158":9,"1583":4,"1585":3,"1586":7,"1596":1,"16":19,"160":10,"1

# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. 
3. Submit your .ipynb and app.py file in your github repository. The dataset is optional to post since it has big size to post. 

